# 1. Import libraries

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# 2. Data loading and cleaning

A spectrum is multivariable. In this project, each spectrum/row is made up of 125 variables/wavelengths/columns.
In the spectral-data-containing Excel sheets being imported, the first column lists the name of each spectrum/row. The first row lists the 125 wavelengths that make up each spectrum. Each sheet corresponds to each storage day of each sample (referred to as condition in the code).

In [15]:
# Load the data from the data-containing Excel file
file_name = 'Data for PCA.xlsx' # Replace with appropriate file name saved in the same folder as the current code
sheets = pd.read_excel(file_name, sheet_name=None, engine='openpyxl')

# If needed, define specific sheets to include. Otherwise comment out this line
selected_sheets = ["Condition 1 Day 0", "Condition 2 Day 0", "Condition 3 Day 0"]

# Extract column headers as wavelengths, group data from all sheets and create labels for each row according to the sheet it came from
group_labels = []
group_data = []
wavelengths = None
for sheet_name, data in sheets.items():
    if sheet_name in selected_sheets: # If selecting all sheets, comment out this line and adjust the corresponding indentation
        if wavelengths is None:
            wavelengths = data.columns.values[1:].astype(float)  # Extract column headers as wavelengths
        cleaned_data = data.iloc[1:, 1:].apply(pd.to_numeric, errors='coerce').dropna()
        group_data.append(cleaned_data.values.astype(float))
        group_labels.extend([sheet_name] * cleaned_data.shape[0])

raw_data = np.vstack(group_data)

# 3. Data preprocessing

Three data preprocessing methods were used successively. This was decided after some trial and error. The eliminated methods are not shown here for the sake of simplicity.

1. MSC (Multiplicative Scatter Correction) is a column-wise method. It first computes the mean spectrum of all spectra, then fits a polynomial between the mean spectrum and each original spectrum, and finally remove the resulting additive and multiplicative components from each original spectrum.

2. Savitzky–Golay smoothing (Savgol smoothing) fits a polynomial to each msc-treated spectrum in each spectral window of a specified length and computes the derivative of the centre point.

3. Standardisation was applied as the final step.

In [16]:
# Define preprocessing functions
def compute_msc(input_data):
    mean_spectrum = np.mean(input_data, axis=0)
    corrected_data = np.zeros_like(input_data)
    for i in range(input_data.shape[0]):
        fit = np.polyfit(mean_spectrum, input_data[i, :], 1, full=True)
        corrected_data[i, :] = (input_data[i, :] - fit[0][1]) / fit[0][0]
    return corrected_data

def compute_savgol_smoothing(input_data, window_length=11, polyorder=2, deriv=2):
    return savgol_filter(input_data, window_length, polyorder, deriv=deriv, axis=1)

# Apply data preprocessing functions and Standardisation
msc_data = compute_msc(raw_data)
sgf2_msc_data = compute_savgol_smoothing(msc_data)
std_sgf2_msc_data = StandardScaler().fit_transform(sgf2_msc_data)

processed_datasets = {
    'MSC + SGF2 + Std': std_sgf2_msc_data
}

# 4. Define colour and marker mappings for each condition and day for PC score plots in Step 5

For context: 
Conditions 1 and 2 had Days 0, 1, 2, 5, 16, 23, 68, while
Conditions 3 had Days 0, 1, 4, 49

Hence, Days 4 & 5 and Days 49 & 68 were made the same colours to allow for rough comparison to the closest day


In [17]:
day_colors = {
    '0': '#1f77b4',  # Blue
    '1': '#17becf',  # Cyan
    '2': '#bcbd22',  # Olive
    '4': '#ff7f0e',  # Orange
    '5': '#ff7f0e',  # Orange
    '16': '#d62728', # Red
    '23': '#e377c2', # Pink
    '49': '#9467bd', # Purple
    '68': '#9467bd', # Purple
}

condition_markers = {
    'Condition 1': 's',  # Square
    'Condition 2': 'x',  # Cross
    'Condition 3': '^', # Triangle
}

# 5. Define a function which performs PCA, exports values to Excel and produces PC score plots

This function 
(i)   performs PCA, 
(ii)  computes the variances explained by each of the top 3 PCs,
(iii) computes the PC loadings of each wavelength,
(iv)  computes the scores of the top 3 PCs of each spectrum.

This function also exports to Excel 
1. the variances explained by each of the top 3 PCs,
2. loadings of each wavelength for each of the top 3 PCs, in ascending order of the wavelengths.

This function also plots: PC1 vs PC2 score, PC2 vs PC3 score, and PC1 vs PC3 score, for all spectra, marking the spectra according to their sample type and storage day.
Example plots for (a) data of all 3 conditions Day 0 only and (b) data of all 3 conditions all days, have been uploaded to the repository.

In [18]:
# Initialise an Excel writer. Rename file accordingly
writer = pd.ExcelWriter('PCA_v1.xlsx', engine='openpyxl')

# Define the PCA function
def perform_pca_export_plot(data_name, data):
    # Perform PCA and extract variances explained by top 3 PCs and save them to Excel
    pca = PCA(n_components=3)
    principal_components = pca.fit_transform(data)
    variances_df = pd.DataFrame([pca.explained_variance_ratio_], columns=[f'PC{i+1}' for i in range(3)])
    variances_df.to_excel(writer, sheet_name=f'{data_name} Variances', index=False)

    # Extract PC1-PC3 loadings for each wavelength and sort in ascending order of the wavelengths
    loadings = pca.components_[:3, :].T 
    sorted_indices = np.argsort(wavelengths)
    sorted_wavelengths = wavelengths[sorted_indices]
    sorted_pc1 = loadings[:, 0][sorted_indices]
    sorted_pc2 = loadings[:, 1][sorted_indices]
    sorted_pc3 = loadings[:, 2][sorted_indices]

    # Save sorted PC1-3 loadings to Excel
    pc_scores_df = pd.DataFrame({
        'Wavelength': sorted_wavelengths,
        'PC1 Loadings': sorted_pc1,
        'PC2 Loadings': sorted_pc2,
        'PC3 Loadings': sorted_pc3
    })
    pc_scores_df.to_excel(writer, sheet_name=f'{data_name} PC Loadings', index=False)

    # Plot 3 score plots
    fig, axes = plt.subplots(1, 3, figsize=(20, 6), dpi=600)
    unique_groups = np.unique(group_labels)
    legend_entries = {}
    color_legend_entries = {}

    for i, (x, y) in enumerate([(0, 1), (1, 2), (0, 2)]):
        ax = axes[i]
        for group in unique_groups:
            condition = 'Condition 1' if 'Condition 1' in group else ('Condition 2' if 'Condition 2' in group else 'Condition 3')
            marker = condition_markers.get(condition, 'o')
            indices = np.where(np.array(group_labels) == group)
            day = group.split()[-1]  # Extract day from group label
            color = day_colors.get(day, '#000000')  # Default to black if day not found
            ax.scatter(principal_components[indices, x], principal_components[indices, y],
                       s=40,
                       c=color,
                       marker=marker,
                       alpha=0.7,
                       label=group)
            if condition not in legend_entries:
               legend_entries[condition] = plt.Line2D([0], [0], marker=marker, color='w', markerfacecolor='none', markeredgecolor='black', markersize=7, label=condition)
            if day not in color_legend_entries:
               color_legend_entries[int(day)] = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=7, label=f'Day {day}')  # Only the day number, if would not like the word day, change to label=day
               # Sort color_legend_entries by numerical day values
               sorted_color_legend_entries = {k: v for k, v in sorted(color_legend_entries.items())}
        ax.set_xlabel(f'PC{x+1} ({pca.explained_variance_ratio_[x]*100:.2f}%)', fontsize=13, family='Arial', color='black')  # Add variance explained
        ax.set_ylabel(f'PC{y+1} ({pca.explained_variance_ratio_[y]*100:.2f}%)', fontsize=13, family='Arial', color='black')  # Add variance explained
        ax.grid(False)
        ax.grid(False)
        ax.axhline(0, color='#BFBFBF', linestyle='--', linewidth=0.5)
        ax.axvline(0, color='#BFBFBF', linestyle='--', linewidth=0.5)
        
        # Customise ticks, font size, colour
        ax.tick_params(axis='both', labelsize=12, labelcolor='black')
        # Set grey border color
        ax.spines['top'].set_color('#A6A6A6')
        ax.spines['right'].set_color('#A6A6A6') 
        ax.spines['left'].set_color('#A6A6A6') 
        ax.spines['bottom'].set_color('#A6A6A6') 
        
    # Customise position of legend
    fig.legend(handles=list(legend_entries.values()) + list(sorted_color_legend_entries.values()), loc='upper right', bbox_to_anchor=(1.25, 1))
    plt.tight_layout()
    plt.savefig(f'{data_name}_PCA_Score_Plots.png', bbox_inches='tight')
    plt.close()

# Execute the function
for name, dataset in processed_datasets.items():
    perform_pca_export_plot(name, dataset)

writer.close()